<a href="https://www.kaggle.com/drjohnwagner/heart-disease-prediction-with-xgboost?scriptVersionId=85327390" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import json
import random
import numpy as np
import pandas as pd
from igraph import Graph
import igraph
from pprint import pprint
#
import plotly.io as pio
import plotly.graph_objects as go
#
from xgboost import XGBClassifier
from xgboost import XGBModel
from xgboost import Booster
#
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle

try:
    from PDSUtilities.xgboost import plot_importance
    from PDSUtilities.xgboost import plot_tree
except ImportError as e:
    try:
        !pip install PDSUtilities --upgrade
        from PDSUtilities.xgboost import plot_importance
        from PDSUtilities.xgboost import plot_tree
    except ImportError as e:
        raise ImportError("You must install PDSUtilities to plot importance and trees...") from e

from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk("/kaggle/input"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# # Colorblindness friendly colours...
# # It is important to make our work
# # as accessible as possible...
# COLORMAP = ["#005ab5", "#DC3220"]
# Labels for plotting...
LABELS = {
    "Sex": "Sex",
    "Age": "Age",
    "MaxHR": "Max HR",
    "OldPeak": "Old Peak",
    "STSlope": "ST Slope",
    "RestingBP": "Rest. BP",
    "FastingBS": "Fast. BS",
    "RestingECG": "Rest. ECG",
    "Cholesterol": "Cholesterol",
    "HeartDisease": "Heart Disease",
    "ChestPainType": "Chest Pain",
    "ExerciseAngina": "Ex. Angina",
}
# Random seed for determinism...
SEED = 395147

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/heart-failure-prediction/heart.csv


## Description
Simple notbook showing how to use `xgboost` to analyse the Heart Disease dataset.

Notebook also shows how to use **`PDSUtilities.xgboost.plot_importance()`** and **`PDSUtilities.xgboost.plot_tree()`**, my Plotly-based replacements for `xgboost.plot_importance()` and `xgboost.plot_tree()`.

Check them out at [https://github.com/DrJohnWagner/PDSUtilities](https://github.com/DrJohnWagner/PDSUtilities).

## Load the Data

In [2]:
# Loading the data from the csv file...
df = pd.read_csv("/kaggle/input/heart-failure-prediction/heart.csv")
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


## Fix the Column Names
Columns `ST_Slope` and `Oldpeak` do not use the same naming convention as the other columns.

We also convert `HeartDisease` to a categorical variable (`int64` to `int8`)...

In [3]:
# Fix the egregious column naming error...
df = df.rename(columns = {"ST_Slope": "STSlope", "Oldpeak": "OldPeak"})

# Always test these things...
assert len(df["STSlope"]) > 0, "Ruh roh! ST_Slope is still terribly mistaken!"
assert len(df["OldPeak"]) > 0, "Ruh roh! Oldpeak is still terribly mistaken!"

# Convert target to categorical
target = pd.Categorical(df["HeartDisease"])
df["HeartDisease"] = target.codes

print("Datatypes")
print("---------")
print(df.dtypes)

Datatypes
---------
Age                 int64
Sex                object
ChestPainType      object
RestingBP           int64
Cholesterol         int64
FastingBS           int64
RestingECG         object
MaxHR               int64
ExerciseAngina     object
OldPeak           float64
STSlope            object
HeartDisease         int8
dtype: object


## Grab the Column Names

In [4]:
# Break the columns into two groupings...
categorical_columns = [column for column in df.columns if df[column].dtypes == np.object]
numerical_columns   = [column for column in df.columns if df[column].dtypes != np.object]

if "HeartDisease" in numerical_columns:
    numerical_columns.remove("HeartDisease")

assert "HeartDisease" not in numerical_columns, "Ruh roh! HeartDisease is still in numerical_columns!"

print("Categorical Columns: ", categorical_columns)
print("  Numerical Columns: ", numerical_columns)

Categorical Columns:  ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'STSlope']
  Numerical Columns:  ['Age', 'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR', 'OldPeak']


## Randomly Redistribute Missing Values
The columns `RestingBP` and `Cholesterol` have records with value 0. Hypothesising that these
represent missing values, we set them to random values drawn from a normal distribution fit to
the rest of the values in each of those columns.

In [5]:
def set_column_value_to_normal_distribution(df, column, value):
    # Compute the column's mean and standard deviation
    # after removing rows whose column matches value...
    mean_value = df[df[column] != value][column].mean()
    std_value  = df[df[column] != value][column].std()
    # Create a random number generator...
    rng = np.random.default_rng(SEED)
    # Now set the column of those rows to a
    # random sample from a normal distribution...
    df[column] = df[column].apply(
        lambda x : rng.normal(mean_value, std_value) if x == value else x
    )
    return df

df = set_column_value_to_normal_distribution(df, "RestingBP"  , 0)
df = set_column_value_to_normal_distribution(df, "Cholesterol", 0)

# Always test...
assert len(df[df["RestingBP"  ] == 0]) == 0, "Ruh roh! One or more patients has crashed again!"
assert len(df[df["Cholesterol"] == 0]) == 0, "Ruh roh! One or more patients has crashed again!"

## Build the model...

Run this cell if you want to perform a very limited grid search:

In [6]:
PERFORM_GRID_SEARCH = True

Run this cell if you **do not** want to perform grid search:

In [7]:
PERFORM_GRID_SEARCH = False

Change `PERFORM_HUGE_GRID_SEARCH` to `True` if you want to do an extensive grid search.

Be aware: this takes many hours on a four-core CPU!

In [8]:
PERFORM_HUGE_GRID_SEARCH = False

In [9]:
# Split the dataset into training and test...
xt, xv, yt, yv = train_test_split(
    df.drop("HeartDisease", axis = 1),
    df["HeartDisease"],
    test_size = 0.2,
    random_state = 42,
    shuffle = True,
    stratify = df["HeartDisease"]
)

# Define the data preparation, feature
# selection and classification pipeline
pipeline = Pipeline(steps = [
    ("transform", ColumnTransformer(
            transformers = [
                ("cat", OrdinalEncoder(), categorical_columns),
                ("num", MinMaxScaler(), numerical_columns)
            ]
        )
    ),
    ("features", SelectKBest()),
    ("classifier", XGBClassifier(
            objective = "binary:logistic", eval_metric = "auc", use_label_encoder = False
        )
    )
])

if PERFORM_GRID_SEARCH:
    # Define our search space for grid search...
    # Short search over gamma as a quick example...
    search_space = [{
        "classifier__n_estimators": [60],
        "classifier__learning_rate": [0.1],
        "classifier__max_depth": [4],
        "classifier__colsample_bytree": [0.2],
        "classifier__gamma": [i / 10.0 for i in range(3, 7)],
        "features__score_func": [chi2],
        "features__k": [10],
    }]
    if PERFORM_HUGE_GRID_SEARCH:
        # Define our search space for grid search...
        # This is a real search but takes hours...
        search_space = [{
            "classifier__n_estimators": [i*10 for i in range(1, 10)],
            "classifier__learning_rate": [0.01, 0.05, 0.1, 0.2],
            "classifier__max_depth": range(1, 10, 1),
            "classifier__colsample_bytree": [i/20.0 for i in range(7)],
            "classifier__gamma": [i / 10.0 for i in range(3, 7)],
            "features__score_func": [chi2],
            "features__k": [10],
        }]
    # Define grid search...
    grid = GridSearchCV(
        pipeline,
        param_grid = search_space,
        # Define cross validation...
        cv = KFold(n_splits = 10, random_state = 917, shuffle = True),
        # Define AUC and accuracy as score...
        scoring = {
            "AUC": "roc_auc",
            "Accuracy": make_scorer(accuracy_score)
        },
        refit = "AUC",
        verbose = 1,
        n_jobs = -1
    )
    # Fit grid search
    grid_model = grid.fit(xt, yt)
    yp = grid_model.predict(xv)
    #
    print(f"Best AUC Score: {grid_model.best_score_}")
    print(f"Accuracy: {accuracy_score(yv, yp)}")
    print("Confusion Matrix: ", confusion_matrix(yv, yp))
    print("Best Parameters: ", grid_model.best_params_)

In [10]:
# Use the new best parameters if they were computed
# else use the previously computed best parameters.
# These produced an AUC score of 0.9244531360448315
# and an accuracy of 0.8858695652173914.
parameters = {
    "classifier__colsample_bytree": 0.2,
    "classifier__gamma": 0.4,
    "classifier__learning_rate": 0.1,
    "classifier__max_depth": 4,
    "classifier__n_estimators": 60,
    "features__k": 10,
    "features__score_func": chi2
}
if PERFORM_GRID_SEARCH:
    parameters = grid_model.best_params_

pipeline.set_params(**parameters)

model = pipeline.fit(xt, yt)
yp = model.predict(xv)

print(f"Accuracy: {accuracy_score(yv, yp)}")
print("Confusion Matrix: ", confusion_matrix(yv, yp))
print("Prediction: ", yp)

Accuracy: 0.8858695652173914
Confusion Matrix:  [[73  9]
 [12 90]]
Prediction:  [1 0 1 0 0 0 0 1 0 1 1 0 0 0 0 1 0 1 0 0 0 1 0 0 1 1 0 1 1 0 0 1 0 0 1 1 1
 1 0 1 1 0 1 0 1 1 1 0 1 0 1 0 0 0 1 0 1 0 0 0 0 1 0 1 0 1 1 1 0 0 0 1 1 1
 0 1 1 1 1 1 1 1 1 0 1 0 1 1 0 1 0 1 0 0 1 1 1 1 0 1 1 0 1 0 1 1 1 0 0 1 1
 0 0 1 1 0 1 0 1 1 0 1 1 0 0 1 0 0 1 0 0 0 0 0 1 1 1 0 0 0 1 1 0 1 1 1 1 0
 1 1 1 0 1 0 1 1 0 0 1 1 1 1 0 0 1 0 1 1 0 1 1 0 1 1 1 0 0 0 0 0 1 1 1 0]


## Plotting the Importance and First Five Trees
Here I demonstrate the use of **`PDSUtilities.xgboost.plot_importance()`** and **`PDSUtilities.xgboost.plot_tree()`** on the `xgboost` model above...

In [11]:
classifier = pipeline["classifier"]
trees = [tree for tree in classifier.get_booster()]
# print(pio.templates["presentation"])
features = [LABELS[column] for column in df.drop("HeartDisease", axis = 1).columns]

fig = plot_importance(classifier, features = features)
fig.update_layout(template = "presentation")
fig.update_layout(width = 700, height = 600)
fig.update_layout(
    margin = { 'l': 150 }, #, 'r': 10, 't': 50, 'b': 10 },
)
# This is literally the dumbest thing I've seen in years...
# This puts space between the ticks and tick labels. SMFH.
fig.update_yaxes(ticksuffix = "  ")

fig.show()

booster = classifier.get_booster()
print("Plotting the first five trees:")
for tree in range(0, np.minimum(5, len(trees))):
    title = f"Tree {tree}"
    grayscale = tree % 2 == 0
    edge_labels = { 'Yes/Missing': "Yes" }
    fig = plot_tree(booster, tree, features = features, grayscale = grayscale, edge_labels = edge_labels)
    fig.update_layout(
        margin = { 'l': 10, 'r': 10, 't': 50, 'b': 10 },
        title = { 'text': title, 'x': 0.5, 'xanchor': "center" },
    )
    fig.show()


Plotting the first five trees:
